<div align="center"><a href="https://www.nvidia.com/en-us/deep-learning-ai/education/"><img src="./assets/DLI_Header.png"></a></div>

# Deploying a Model for Inference at Production Scale

## 07 - Metrics
-------

**Table of Contents**

* [Introduction](#introduction)
* [Prometheus](#prometheus)
* [Exercise](#exercise)
* [Docker Compose](#docker-compose)
* [Next Steps](#next-steps)

<a id="introduction"></a>
### Introduction

In this notebook, we will look at some of the server metrics Triton has automatically collected for us. Triton provides Prometheus metrics indicating GPU and request statistics. By default, these metrics are available at http://localhost:8002/metrics. The metrics are only available by accessing the endpoint, and are not pushed or published to any remote server. The metric format is plain text so they can be viewed directly, for example by running the `curl` command below.

In [1]:
!curl triton:8002/metrics

# HELP nv_inference_request_success Number of successful inference requests, all batch sizes
# TYPE nv_inference_request_success counter
nv_inference_request_success{gpu_uuid="GPU-45dcd2ba-97ce-3611-2e50-a5b7da799e59",model="dynamic-batching-model",version="1"} 30000.000000
nv_inference_request_success{model="simple-pytorch-model-cpu",version="1"} 1000.000000
nv_inference_request_success{gpu_uuid="GPU-45dcd2ba-97ce-3611-2e50-a5b7da799e59",model="simple-tensorrt-fp32-model",version="1"} 1.000000
nv_inference_request_success{gpu_uuid="GPU-45dcd2ba-97ce-3611-2e50-a5b7da799e59",model="simple-tensorrt-fp16-model",version="1"} 10301.000000
nv_inference_request_success{gpu_uuid="GPU-45dcd2ba-97ce-3611-2e50-a5b7da799e59",model="simple-onnx-model",version="1"} 1.000000
nv_inference_request_success{gpu_uuid="GPU-45dcd2ba-97ce-3611-2e50-a5b7da799e59",model="simple-pytorch-model",version="1"} 1001.000000
# HELP nv_inference_request_failure Number of failed inference requests, all batch sizes
# TYP

<a id="prometheus"></a>
### Prometheus

For a format that's more pleasing on the eyes, Triton metrics are compatible with [Prometheus](https://prometheus.io/). Copy and paste your JupyterLab URL into the code cell below to generate a link to the Prometheus window. There will be a screen that looks like this:

<div align="center"><img src="./assets/Prom.png"></div>

In [3]:
import IPython
my_url = "http://ec2-54-209-80-90.compute-1.amazonaws.com/lab/lab/workspaces/auto-7/tree/07_Metrics.ipynb"
prometheus_url = my_url.rsplit(".com", 1)[0] + ".com:9090/graph"
prometheus_url

'http://ec2-54-209-80-90.compute-1.amazonaws.com:9090/graph'

Let's see if we're pushing our hardware too much by looking at the GPU utilization. This could help us diagnose potential crashes or we could set up an [alert](https://prometheus.io/docs/alerting/latest/overview/) if we're close to 100% and need more resources.

Click the earth icon, and select `nv_gpu_utilization`. Then, click "Execute".

<div align="center"><img src="./assets/Prom_Add_Metric.png"></div>

This will produce a line that looks like this:

*nv_gpu_utilization{gpu_uuid="GPU-76eed5e4-a509-ea60-8ce3-5c9b82f9252b", instance="triton:8002", job="prometheus"}*

Click the `Graph` tab, and a graph will appear with the GPU utilization over the past hour.

<a id="exercise"></a>
### Exercise #3 - Make your own dashboard

Other metrics can be compared side by side. Scroll down to the bottom of the Prometheus window and click the "Add Panel" button on the left. This allows us to repeat the process above. Add a few more metrics of your choosing and analyze your previous activity. Can you recognize when you sent requests to Triton from the previous notebooks?

<a id="docker-compose"></a>
### Docker Compose

One of the most straightforward ways of setting up both Triton and Prometheus is with a tool called [Docker Compose](https://docs.docker.com/compose/). It allows us to deploy multiple [Docker containers](https://www.docker.com/resources/what-container) that can share data and other resources. We highly recommend learning [Docker basics](https://www.docker.com/101-tutorial) before proceeding.

For now, let's focus on `triton` and break down each of the keys:
* [command](https://docs.docker.com/compose/compose-file/compose-file-v2/#command): The command for the container to run once it's built. In this case, we're running the command to initiate the server if we had installed the Triton Inference Server Library locally as [described here](https://docs.nvidia.com/deeplearning/triton-inference-server/user-guide/docs/quickstart.html#run-triton-inference-server).
* [image](https://docs.docker.com/compose/compose-file/compose-file-v2/#image): The base image that we're building off of, in this case, the [Triton Inference Server](https://catalog.ngc.nvidia.com/orgs/nvidia/containers/tritonserver/tags) image.
* [shm-size](https://docs.docker.com/compose/compose-file/compose-file-v2/#shm_size): The amount of memory to share with the container. In this case, we're giving it 1 gigabyte for faster computation.
* [ulimits](https://docs.docker.com/compose/compose-file/compose-file-v2/#ulimits): The max number of open file descriptors per process explained in this [Stack Overflow](https://stackoverflow.com/questions/24955883/what-is-the-max-opened-files-limitation-on-linux) post.
* [ports](https://docs.docker.com/compose/compose-file/compose-file-v2/#ports): The ports to expose from the container.
* [volumes](https://docs.docker.com/compose/compose-file/compose-file-v2/#volume-configuration-reference): A directory that can be shared between a container and it's host.

Below is the a Docker compose file very similar to what we used to set up this course.

In [2]:
import IPython
IPython.display.Code(filename="assets/docker-compose.yml", language="yaml") 

version: "2.3"

volumes:
  # Allows us to write files to Triton from JupyterLab
  model_repository:

services:
  #Each service is a Docker container. Below is how to pull Triton from NGC.
  triton:
    command: "tritonserver --model-repository=/models --exit-on-error=false --model-control-mode=poll --repository-poll-secs=30"
    image: nvcr.io/nvidia/tritonserver:20.12-py3
    runtime: nvidia
    ipc: host
    shm_size: "1g"
    ulimits:
      memlock: -1
      stack: 67108864
    ports:
      - 8000:8000
      - 8001:8001
      - 8002:8002
    volumes:
      - model_repository:/models

  # This is the JupyterLab environment. We build it with a Dockerfile.
  lab:
    build: .
    runtime: nvidia
    environment:
      - JUPYTER_TOKEN

    volumes:
      - ./task/:/dli/task/
      - model_repository:/dli/task/models

  # Prometheus
  prometheus:
    image: docker.io/prom/prometheus:latest
    volumes:
      - ./prometheus.yml:/prometheus.yml
    command:
      - "--config.file=/prometheus.yml"
    ports:
      - 9090:9090

  # Nginx is how we make the container available to sudents over the internet.
  nginx:
    image: nginx:1.15.12-alpine
    volumes:
      - ./nginx.conf:/etc/nginx/nginx.conf

Next, let's focus on the `prometheus` service. It's simpler than the `triton` service, but we still need to provide it with a [configuration file](https://prometheus.io/docs/prometheus/latest/configuration/configuration/). Here are some of the keys.

* `global`: Defines properties we want to add to every  Prometheus job.
  * `scrape_interval`: How often a job pulls from a data source
  * `external_labels`: The labels to add to any tine series or alert
* [scrape_configs](https://prometheus.io/docs/prometheus/latest/configuration/configuration/#scrape_config): Specifies a target and how to interact with it.

In this case, we're telling it to pull information from `triton` (as defined in the Docker Compose file above) metrics every 5 seconds. We can give this process a `job_name` to make it easier to identify.

In [3]:
IPython.display.Code(filename="assets/prometheus.yml", language="yaml") 

global:
  scrape_interval: 15s
  external_labels:
    monitor: 'codelab-monitor'

scrape_configs:
  - job_name: 'prometheus'
    scrape_interval: 5s

    static_configs:
      - targets: ['triton:8002']

For additional resources on how to utilize the Triton Inference Server's metrics, please consult:

* [Triton Inference Server Metrics Documentation](https://github.com/triton-inference-server/server/blob/r20.12/docs/metrics.md)
* [Saleforce Blogpost on Benchmarking Triton Inference Server](https://blog.einstein.ai/benchmarking-tensorrt-inference-server/)


<a id="next-steps"></a>
### Next Steps

As this online environment already has a Triton Inference Server deployed, our last challenge to you is setup Triton on your own hardware. We hope the above resources can provide a starting point. For more information, please check out the [Getting Started Guide](https://github.com/triton-inference-server/server/blob/r21.11/docs/quickstart.md).

Good luck!

<div align="center"><a href="https://www.nvidia.com/en-us/deep-learning-ai/education/"><img src="./assets/DLI_Header.png"></a></div>